In [1]:
# Install OpenAI library
!pip install -U -q openai tenacity

In [2]:
!pip install openai==0.28

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [4]:
import json
import pandas as pd
import openai

# Load OpenAI API key from file
with open("/content/OPENAI_API_KEY.txt", "r") as f:
    openai_api_key = f.read().strip()

openai.api_key = openai_api_key  # Set OpenAI API key

# Load weather data from CSV file
weather_data_file = "/content/top100cities_weather_data.csv"
weather_df = pd.read_csv(weather_data_file)
# weather_df = weather_df[[
#     'country',
#     'location_name',
#     'latitude',
#     'longitude',
#     'timezone',
#     'temperature_celsius',
#     'condition_text',
#     'humidity'
# ]]

weather_df.head(10)

,City,Temperature (Celsius),Wind Speed (m/s),Latitude,Longitude,Description,Country
0,Paris,10.46,2.06,48.8534,2.3488,clear sky,France
1,Dubai,29.32,3.09,25.2582,55.3047,clear sky,United Arab Emirates
2,Madrid,9.97,3.60,40.4165,-3.7026,clear sky,Spain
3,Tokyo,21.58,2.06,35.6895,139.6917,broken clouds,Japan
4,Amsterdam,8.00,3.58,52.3740,4.8897,clear sky,Netherlands
5,Berlin,15.66,2.68,52.5244,13.4105,clear sky,Germany
6,Rome,26.09,3.60,34.2570,-85.1647,broken clouds,Italy
7,New York City,21.79,4.63,40.7143,-74.0060,clear sky,United States
8,Barcelona,14.23,4.12,41.3888,2.1590,shower rain,Spain
9,London,7.67,2.57,51.5085,-0.1257,scattered clouds,United Kingdom


In [5]:
weather_df.rename(columns = {'Temperature (Celsius)':'temperature_celsius','Wind Speed (m/s)':'windspeed'}, inplace = True)
weather_df.head(10)

,City,temperature_celsius,windspeed,Latitude,Longitude,Description,Country
0,Paris,10.46,2.06,48.8534,2.3488,clear sky,France
1,Dubai,29.32,3.09,25.2582,55.3047,clear sky,United Arab Emirates
2,Madrid,9.97,3.60,40.4165,-3.7026,clear sky,Spain
3,Tokyo,21.58,2.06,35.6895,139.6917,broken clouds,Japan
4,Amsterdam,8.00,3.58,52.3740,4.8897,clear sky,Netherlands
5,Berlin,15.66,2.68,52.5244,13.4105,clear sky,Germany
6,Rome,26.09,3.60,34.2570,-85.1647,broken clouds,Italy
7,New York City,21.79,4.63,40.7143,-74.0060,clear sky,United States
8,Barcelona,14.23,4.12,41.3888,2.1590,shower rain,Spain
9,London,7.67,2.57,51.5085,-0.1257,scattered clouds,United Kingdom


In [15]:
import json
import pandas as pd
import openai

# Load OpenAI API key from file
with open("/content/OPENAI_API_KEY.txt", "r") as f:
    openai_api_key = f.read().strip()

openai.api_key = openai_api_key  # Set OpenAI API key

# Function to load the weather data
def load_weather_data():
    global weather_df
    weather_data_file = "/content/top100cities_weather_data.csv"
    weather_df.rename(columns = {'Temperature (Celsius)':'temperature_celsius','Wind Speed (m/s)':'windspeed'}, inplace = True)

def get_current_weather(latitude, longitude, tolerance=0.05):
    """Get the current weather from the local CSV file for a given latitude and longitude."""

    # Reload weather_df within the function to ensure it's the latest data
    load_weather_data()

    latitude = float(latitude)
    longitude = float(longitude)

    print(f"Searching for weather data at Latitude: {latitude}, Longitude: {longitude}")

    # Check for coordinates within the given tolerance
    weather_info = weather_df[
        (weather_df['Latitude'].between(latitude - tolerance, latitude + tolerance)) &
        (weather_df['Longitude'].between(longitude - tolerance, longitude + tolerance))
    ]

    if weather_info.empty:
        print("Available weather data coordinates:")
        print(weather_df[['Latitude', 'Longitude']].drop_duplicates())  # Debugging line
        return json.dumps({
            "error": f"Sorry, we couldn't find weather data for the specified location (Latitude: {latitude}, Longitude: {longitude}). Please try again."
        })

    result = {
        "latitude": weather_info['Latitude'].values[0],
        "longitude": weather_info['Longitude'].values[0],
        "temperature": weather_info['temperature_celsius'].values[0],
        "humidity": weather_info['windspeed'].values[0],
        "condition": weather_info['Description'].values[0],
        "country": weather_info['Country'].values[0],
        "location_name": weather_info['City'].values[0],
    }
    return json.dumps(result)


def run_conversation(content):
    # Initialize fresh message list for each conversation
    messages = [{"role": "user", "content": content}]

    tools = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given latitude and longitude from a top100cities_weather_data.csv CSV file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "string",
                        "description": "The latitude of a place",
                    },
                    "longitude": {
                        "type": "string",
                        "description": "The longitude of a place",
                    },
                },
                "required": ["latitude", "longitude"],
            },
        }
    ]

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            functions=tools,
            function_call="auto",
        )

        if not response or 'choices' not in response or not response['choices']:
            print("No valid response received from the API.")
            return None

        response_message = response['choices'][0]['message']
        tool_call = response_message.get("function_call")

        if tool_call:
            messages.append(response_message)
            available_functions = {
                "get_current_weather": get_current_weather,
            }

            function_name = tool_call["name"]
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call["arguments"])

            # Retrieve exact latitude and longitude
            latitude = function_args.get("latitude")
            longitude = function_args.get("longitude")

            function_response = function_to_call(
                latitude=latitude,
                longitude=longitude,
            )
            print(f"API: {function_response}")

            messages.append(
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                }
            )

            second_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=messages
            )
            return second_response['choices'][0]['message']['content']

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


if __name__ == "__main__":
    question = "What's the weather like in Seoul?"
    response = run_conversation(question)

    if response:
        print("Assistant response:", response)
    else:
        print("No response received.")


Searching for weather data at Latitude: 37.5665, Longitude: 126.978
API: {"latitude": 37.5683, "longitude": 126.9778, "temperature": 15.41, "humidity": 2.06, "condition": "clear sky", "country": "South Korea", "location_name": "Seoul"}
Assistant response: The weather in Seoul is currently 15.41°C with a clear sky.


In [16]:
if __name__ == "__main__":
    question = "What's the weather like in Madrid?"
    response = run_conversation(question)

    if response:
        print("Assistant response:", response)
    else:
        print("No response received.")

Searching for weather data at Latitude: 40.4168, Longitude: -3.7038
API: {"latitude": 40.4165, "longitude": -3.7026, "temperature": 9.97, "humidity": 3.6, "condition": "clear sky", "country": "Spain", "location_name": "Madrid"}
Assistant response: The weather in Madrid is currently 9.97°C with a clear sky. The humidity is at 3.6%.


In [17]:
if __name__ == "__main__":
    question = "What's the weather like in Osaka?"
    response = run_conversation(question)

    if response:
        print("Assistant response:", response)
    else:
        print("No response received.")

Searching for weather data at Latitude: 34.6937, Longitude: 135.5023
API: {"latitude": 34.6937, "longitude": 135.5022, "temperature": 19.0, "humidity": 1.54, "condition": "light rain", "country": "Japan", "location_name": "Osaka"}
Assistant response: The weather in Osaka is currently 19.0°C with light rain.
